In [ ]:
import time

class State:
    def __init__(self, board=None, turn=1):
        # 돌의 배치
        self.board = board if board is not None else [0]*9
        self.turn = turn  # -1이면 컴퓨터가 먼저, 1이면 사용자가 먼저gg

    def piece_count(self, pieces):
        count = 0
        for i in pieces:
            if i == 1:
                count += 1
        return count

    def is_win(self):
        for line in [(0, 1, 2), (3, 4, 5), (6, 7, 8),  # Rows
                     (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Columns
                     (0, 4, 8), (2, 4, 6)]:            # Diagonals
            if all(self.board[i] == 1 for i in line):
                return True
        return False

    def is_lose(self):
        for line in [(0, 1, 2), (3, 4, 5), (6, 7, 8),  # Rows
                     (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Columns
                     (0, 4, 8), (2, 4, 6)]:            # Diagonals
            if all(self.board[i] == -1 for i in line):
                return True
        return False

    def is_draw(self):
        return all(cell != 0 for cell in self.board) and not self.is_win() and not self.is_lose()

    def is_done(self):
        return self.is_win() or self.is_lose() or self.is_draw()

    def next(self, action):
        new_board = self.board[:]
        new_board[action] = 1 if self.turn == 1 else -1
        # 턴을 변경하여 새 상태 반환
        return State(new_board, -self.turn)

    def legal_actions(self):
        return [i for i in range(9) if self.board[i] == 0]

    def is_first_player(self):
        return self.turn == 1  # 사용자가 먼저인 경우

    def render(self):
        symbols = {1: 'o', -1: 'x', 0: '-'}
        for i in range(9):
            print(symbols[self.board[i]], end='')
            if i % 3 == 2:
                print()

In [ ]:
board = [0,1,1,
         0,1,-1,
         1,1,1]
st = State(board)
st.render()

-oo
-ox
ooo


In [ ]:
import random
import math
import copy

class Node:
    def __init__(self, position=None, p=None, state=None):
        self.position = position # 현재 노드에서 선택된 위치
        self.p = p # 부모 노드
        self.state = state # 상태
        self.player = state.turn # 현재 플레이어
        self.score = 0 # 시뮬레이션 결과의 합
        self.v = 0 # 방문횟수
        self.child = [] # 자식 노드
        self.empty_positions = state.legal_actions() # 가능한 이동 위치

    def add_child(self, pos, state): # 새 노드 만드는 경우
        new_state = state.next(pos)
        node = Node(pos, self, new_state)
        self.empty_positions.remove(pos)
        self.child.append(node)
        return node

    def get_max_uct_index(self, ucts): # 가장 큰 UCT 값을 가진 인덱스 찾기
        index = 0
        max_uct = ucts[0]
        for i, uct in enumerate(ucts[1:], 1):
            if max_uct < uct:
                index = i
                max_uct = uct
        return index

    def get_uct_value(self): # 현재 노드에서 자식 노드들의 UCT 값 계산
        ucts = []
        for node in self.child:
            if node.v > 0:
                uct = node.score / node.v + math.sqrt(1.1 * math.log(self.v) / node.v)
            else:
                uct = float('inf')  # 방문하지 않은 노드는 우선 선택
            ucts.append(uct)
        index = self.get_max_uct_index(ucts)
        return self.child[index]

    def __lt__(self, other): # 두 노드 비교(방문 횟수 적은 노드 선택)
        return self.v < other.v


class MCTS:
    def __init__(self):
        self.state = None
        self.node = None

    def mcts(self, state, iterate_number):
        root_node = Node(state=state)
        if len(root_node.empty_positions) == 1:
            return root_node.empty_positions[0]

        for _ in range(iterate_number): # 트리 탐색 반복
            self.state = copy.deepcopy(state)
            self.node = root_node
            self.selection()
            result = self.simulation()
            self.update_node(result)

        for n in root_node.child:
            print(f"Position: {n.position + 1}, Score: {n.score}, Visits: {n.v}")
        return self.get_best_position(root_node)

    def get_best_position(self, node):
        if not node.child:
            return random.choice(node.empty_positions)
        node.child.sort()  # 방문 횟수 기준으로 정렬
        return node.child[-1].position

    def selection(self): # UCT 값이 최대인 자식 노드 선택
        while not self.node.empty_positions and self.node.child:
            self.node = self.node.get_uct_value()
            self.state = self.node.state

    def simulation(self): # 승패 예측하는 시뮬레이션
        while not self.state.is_done():
            pos = random.choice(self.state.legal_actions())
            self.state = self.state.next(pos)
        if self.state.is_win():
            return 1  # 컴퓨터 승
        elif self.state.is_lose():
            return -1  # 사용자 승
        else:
            return 0  # 무승부

    def update_node(self, result):
        while self.node is not None:
            self.node.score += result
            self.node.v += 1
            self.node = self.node.p


In [ ]:
import time

def play_game():
    # 게임 시작 시 사용자에게 차례 선택
    print("Do you want to go first or let the computer go first?")
    choice = input("Type '1' for you first or '-1' for computer first: ").strip()

    if choice == '1':
        state = State(turn=1)  # 사용자가 먼저
    elif choice == '-1':
        state = State(turn=-1)  # 컴퓨터가 먼저
    else:
        print("Invalid choice, defaulting to you first.")
        state = State(turn=1)

    mcts = MCTS()

    while not state.is_done():
        state.render()
        if state.is_first_player():
            # 사용자의 차례
            print("Your turn! Choose a position (0-8): ")
            action = int(input())
            if action not in state.legal_actions():
                print("잘못된 입력입니다. 다시 선택해주세요")
                continue
        else:
            # 컴퓨터의 차례
            print("Computer's turn...")
            start_time = time.time()
            action = mcts.mcts(state, iterate_number=10000)  # MCTS를 사용해 최적의 행동 찾기
            end_time = time.time()
            print(f"Computer took {end_time - start_time:.2f} seconds to decide.")

        # 상태 업데이트
        state = state.next(action)

    # 결과 출력
    state.render()
    if state.is_win():
        print("You win!" if state.is_first_player() else "Computer wins!")
    elif state.is_lose():
        print("Computer wins!" if state.is_first_player() else "You win!")
    else:
        print("It's a draw!")


In [ ]:
play_game()

Do you want to go first or let the computer go first?
Type '1' for you first or '-1' for computer first: -1
---
---
---
Computer's turn...
Computer took 2.19 seconds to decide.
---
---
--x
Your turn! Choose a position (0-8): 
0
o--
---
--x
Computer's turn...
Computer took 1.10 seconds to decide.
o-x
---
--x
Your turn! Choose a position (0-8): 
5
o-x
--o
--x
Computer's turn...
Computer took 1.75 seconds to decide.
o-x
--o
-xx
Your turn! Choose a position (0-8): 
6
o-x
--o
oxx
Computer's turn...
Computer took 0.77 seconds to decide.
oxx
--o
oxx
Your turn! Choose a position (0-8): 
4
oxx
-oo
oxx
Computer's turn...
Computer took 0.00 seconds to decide.
oxx
xoo
oxx
It's a draw!
